In [ ]:
from pathlib import Path

import polars as pl
import yaml

In [9]:
root = Path("~/Data/foundata/VISTA")

In [49]:
def preprocess_hhs(hhs, config: dict):
    column_mapping = config["column_mappings"]
    ownership_mapping = config["home_ownership"]["default"]
    zone_mapping = config["zone"]["default"]

    hhs = hhs.select(column_mapping.keys()).rename(column_mapping)

    hhs = hhs.with_columns(pl.col("year").str.slice(0, 4).cast(pl.Int32).alias("year"))

    # income
    hhs = hhs.with_columns(
        pl.when(pl.col("hh_income").is_null())
        .then(0)
        .otherwise(
            (pl.col("hh_income").str.slice(1).str.replace_all(",", "").cast(pl.Int32))
            * 52
            * 0.6  # weekly to annual and AUS $ to EURO
        )
        .alias("hh_income")
    )

    # ownership
    hhs = hhs.with_columns(
        pl.col("home_ownership")
        .replace_strict(ownership_mapping)
        .fill_null("unknown")
        .alias("home_ownership")
    )

    # urban/rural
    hhs = hhs.with_columns(
        pl.col("zone")
        .replace_strict(zone_mapping)
        .fill_null("unknown")
        .alias("urban/rural")
    )
    hhs = hhs.drop("zone")

    # weight
    hhs = hhs.with_columns(
        pl.when(pl.col("wd_weight").is_null())
        .then(pl.col("we_weight"))
        .otherwise(pl.col("wd_weight"))
        .alias("weight")
    )
    hhs = hhs.drop("wd_weight", "we_weight")

    # remove rows with any nulls
    hhs = hhs.drop_nulls()

    return hhs


hhs = pl.read_csv(root / "2012-2020" / "households_vista_2012_2020_lga_v1.csv")
config = yaml.safe_load(open("vista/hh_dictionary.yaml"))

hhs = preprocess_hhs(hhs, yaml.safe_load(open("vista/hh_dictionary.yaml")))

hhs.head()

hid,year,day,month,home_ownership,hh_size,hh_income,num_vehicles,num_bikes,urban/rural,weight
str,i32,str,str,str,i64,f64,i64,i64,str,f64
"""Y12H0000101""",2012,"""Monday""","""May""","""owned""",4,38220.0,2,2,"""suburban""",87.77
"""Y12H0000102""",2012,"""Tuesday""","""May""","""owned""",4,53040.0,3,0,"""suburban""",85.93
"""Y12H0000103""",2012,"""Wednesday""","""May""","""owned""",4,31200.0,2,0,"""suburban""",87.12
"""Y12H0000104""",2012,"""Thursday""","""May""","""owned""",3,58500.0,3,2,"""suburban""",87.24
"""Y12H0000107""",2012,"""Sunday""","""May""","""owned""",3,54600.0,1,3,"""suburban""",261.58


In [ ]:
def preprocess_persons(persons, config: dict):
    column_mapping = config["column_mappings"]
    sex_mapping = config["sex"]["default"]
    relationship_mapping = config["relationship"]["default"]
    has_license_mapping = config["has_licence"]["default"]
    occupation_mapping = config["occupation"]["default"]

    persons = persons.select(column_mapping.keys()).rename(column_mapping)

    # sex
    persons = persons.with_columns(
        pl.col("sex").replace_strict(sex_mapping).alias("sex")
    )

    # relationship
    persons = persons.with_columns(
        pl.col("relationship")
        .replace_strict(relationship_mapping)
        .alias("relationship")
    )

    # has_license
    persons = persons.with_columns(
        pl.col("has_license")
        .replace_strict(has_license_mapping, default=None)
        .alias("has_license")
    )

    # employment
    persons = persons.with_columns(
        pl.when(pl.col("anywork") == "Y")
        .then(pl.lit("employed"))
        .otherwise(
            pl.when(pl.col("studying") == "No Study")
            .then(pl.lit("unemployed"))
            .otherwise(pl.lit("education"))
        )
        .alias("employment_status")
    )
    persons = persons.drop("anywork", "studying")

    # occupation
    persons = persons.with_columns(
        pl.col("occupation").replace_strict(occupation_mapping).alias("occupation")
    )

    return persons


config = yaml.safe_load(open("vista/person_dictionary.yaml"))
columns = list(config["column_mappings"].keys())

persons = pl.read_csv(
    root / "2012-2020" / "persons_vista_2012_2020_lga_v1.csv", columns=columns
)

persons = preprocess_persons(persons, config)

persons.head()

pid,hid,age,sex,relationship,has_license,occupation,employment_status
str,str,i64,str,str,bool,str,str
"""Y12H0000101P01""","""Y12H0000101""",50,"""male""","""self""",true,"""community_personal_service""","""employed"""
"""Y12H0000101P02""","""Y12H0000101""",43,"""female""","""spouse/partner""",true,"""not_in_labor_force""","""unemployed"""
"""Y12H0000101P03""","""Y12H0000101""",11,"""female""","""child""",false,"""not_in_labor_force""","""education"""
"""Y12H0000101P04""","""Y12H0000101""",6,"""female""","""child""",false,"""not_in_labor_force""","""education"""
"""Y12H0000102P01""","""Y12H0000102""",57,"""female""","""spouse/partner""",true,"""not_in_labor_force""","""unemployed"""


In [85]:
years = ["2012-2020", "2022-2023", "2023-2024"]
hhs_names = [
    "households_vista_2012_2020_lga_v1.csv",
    "household_vista_2022_2023.csv",
    "household_vista_2023_2024.csv",
]
persons_names = [
    "persons_vista_2012_2020_lga_v1.csv",
    "person_vista_2022_2023.csv",
    "person_vista_2023_2024.csv",
]

hh_config = yaml.safe_load(open("vista/hh_dictionary.yaml"))
hh_columns = list(hh_config["column_mappings"].keys())
persons_config = yaml.safe_load(open("vista/person_dictionary.yaml"))
persons_columns = list(persons_config["column_mappings"].keys())

for year, hh_name, persons_name in zip(years, hhs_names, persons_names):

    print(year, ":")

    hhs = pl.read_csv(root / year / hh_name, columns=hh_columns)
    hhs = preprocess_hhs(hhs, hh_config)
    persons = pl.read_csv(root / year / persons_name, columns=persons_columns)
    persons = preprocess_persons(persons, persons_config)

    print("number of hhs: ", len(hhs), " & persons: ", len(persons))

2012-2020 :
number of hhs:  30195  & persons:  77428
2022-2023 :


ColumnNotFoundError: unable to find column "hhinc"; valid columns: ["hhid", "surveyperiod", "hhsize", "dwelltype", "owndwell", "totalbikes", "totalvehs", "travmonth", "travdow", "homelga", "youngestgroup_5", "aveagegroup_5", "oldestgroup_5", "hhinc_group", "hhpoststratweight", "hhpoststratweight_GROUP_1", "hhpoststratweight_GROUP_2", "hhpoststratweight_GROUP_3", "hhpoststratweight_GROUP_4", "hhpoststratweight_GROUP_5", "hhpoststratweight_GROUP_6", "hhpoststratweight_GROUP_7", "hhpoststratweight_GROUP_8", "hhpoststratweight_GROUP_9", "hhpoststratweight_GROUP_10", "homesubregion_ASGS", "homeregion_ASGS", "dayType"]